In [1]:
# Load data
# Clean data
# Feature Enginnering
# Visualizing
# Modelling
# Predict to test data
# Submit

In [2]:
import torch,torchvision
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import wandb
from ray import tune

In [3]:
# !unzip ./data/titanic\ \(7\).zip

In [4]:
test_df = pd.read_csv('./test.csv')
data = pd.read_csv('./train.csv')
sample_submission = pd.read_csv('./gender_submission.csv')

In [5]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
data['Age'] = data['Age'].fillna(data['Age'].mean())

In [8]:
data.drop('Cabin',axis=1,inplace=True)

In [9]:
data['Age'] = data['Age'].fillna(data['Age'].mean())

In [10]:
data.dropna(inplace=True)

In [11]:
data.drop('Name',axis=1,inplace=True)

In [12]:
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,female,35.0,1,0,113803,53.1000,S
4,5,0,3,male,35.0,0,0,373450,8.0500,S


In [13]:
tickets = []
for ticket in data['Ticket']:
    ticket_splited = ticket.split(' ')
    try:
        ticket = ticket_splited[1]
    except:
        ticket = ticket_splited[0]
    try:
        tickets.append(int(ticket))
    except:
        try:
            ticket = ticket_splited[2]
        except:
            ticket = np.mean(tickets)
        tickets.append(int(ticket))

In [14]:
len(tickets)

889

In [15]:
len(data)

889

In [16]:
data['Ticket'] = tickets

In [17]:
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,male,22.0,1,0,21171,7.2500,S
1,2,1,1,female,38.0,1,0,17599,71.2833,C
2,3,1,3,female,26.0,0,0,3101282,7.9250,S
3,4,1,1,female,35.0,1,0,113803,53.1000,S
4,5,0,3,male,35.0,0,0,373450,8.0500,S


In [18]:
data['Sex'].value_counts()

male      577
female    312
Name: Sex, dtype: int64

In [19]:
labels = {'male':0,'female':1}

In [20]:
sex_all = []
for sex_iter in data['Sex']:
        sex_all.append(labels[sex_iter])

In [21]:
data['Sex'] = sex_all

In [22]:
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,0,22.0,1,0,21171,7.2500,S
1,2,1,1,1,38.0,1,0,17599,71.2833,C
2,3,1,3,1,26.0,0,0,3101282,7.9250,S
3,4,1,1,1,35.0,1,0,113803,53.1000,S
4,5,0,3,0,35.0,0,0,373450,8.0500,S


In [23]:
data['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [24]:
labels = {'S':0,'C':1,'Q':2}

In [25]:
Embarked_all = []

In [26]:
for Embarked_iter in data['Embarked']:
        Embarked_all.append(labels[Embarked_iter])

In [27]:
data['Embarked'] = sex_all

In [28]:
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,0,22.0,1,0,21171,7.2500,0
1,2,1,1,1,38.0,1,0,17599,71.2833,1
2,3,1,3,1,26.0,0,0,3101282,7.9250,1
3,4,1,1,1,35.0,1,0,113803,53.1000,1
4,5,0,3,0,35.0,0,0,373450,8.0500,0


In [29]:
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Sex              int64
Age            float64
SibSp            int64
Parch            int64
Ticket           int64
Fare           float64
Embarked         int64
dtype: object

In [30]:
# Feature Enginner

In [31]:
cols = data.drop(['Survived'],axis=1)

In [32]:
cols = list(cols.columns)

In [33]:
def feature_enginnerg(data,col):
    print(len(data))
    max_no = data[col].quantile(0.99) # 0.95
    min_no = data[col].quantile(0.05)
    data = data[data[col] > min_no]
    data = data[data[col] < max_no]
    print(len(data))
    return data

In [34]:
data_just = data

In [35]:
feature_enginnering_or_not = False

In [36]:
cols = ['PassengerId','Pclass']

In [37]:
if feature_enginnering_or_not:
    for col in cols:
        print('*'*50)
        print(col)
        data = feature_enginnerg(data,col)
        print(col)
        print('*'*50)

In [38]:
# Modelling

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
X,y = data.drop(data[['Survived']],axis=1),data[['Survived']]

In [41]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125)

In [42]:
batch_size = 32
epochs = 100

In [43]:
device = 'cuda'

In [44]:
from tqdm import tqdm

In [45]:
def test(X_test,y_test,model):
    X_test = torch.tensor(np.array(X_test))
    y_test = torch.tensor(np.array(y_test))
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for idx in range(len(X_test)):
            preds = model(X_test[idx].to(device).float().view(1,9))
            results = np.round(y_test[idx].float().cpu()) == np.round(preds.float()[0].cpu())
            if any(results):
                correct += 1
            total += 1
    model.train()
    return round(correct/total,3)

In [46]:
data

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,0,22.000000,1,0,21171,7.2500,0
1,2,1,1,1,38.000000,1,0,17599,71.2833,1
2,3,1,3,1,26.000000,0,0,3101282,7.9250,1
3,4,1,1,1,35.000000,1,0,113803,53.1000,1
4,5,0,3,0,35.000000,0,0,373450,8.0500,0
...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,0,27.000000,0,0,211536,13.0000,0
887,888,1,1,1,19.000000,0,0,112053,30.0000,1
888,889,0,3,1,29.699118,1,2,6607,23.4500,1
889,890,1,1,0,26.000000,0,0,111369,30.0000,0


In [47]:
class Model(nn.Module):
    def __init__(self,starter=64,activation=nn.ReLU()):
        super().__init__()
        self.fc1 = nn.Linear(9,starter)
        self.fc1batchnorm = nn.BatchNorm1d(starter)
        self.fc2 = nn.Linear(starter,starter*2)
        self.fc2batchnorm = nn.BatchNorm1d(starter*2)
        self.fc3 = nn.Linear(starter*2,starter*4)
        self.fc3batchnorm = nn.BatchNorm1d(starter*4)
        self.fc4 = nn.Linear(starter*4,starter*8)
        self.fc4batchnorm = nn.BatchNorm1d(starter*8)
        self.fc5 = nn.Linear(starter*8,starter*16)
        self.fc5batchnorm = nn.BatchNorm1d(starter*16)
        self.fc6 = nn.Linear(starter*16,starter*32)
        self.fc6batchnorm = nn.BatchNorm1d(starter*32)
        self.fc7 = nn.Linear(starter*32,starter*16)
        self.fc7batchnorm = nn.BatchNorm1d(starter*16)
        self.output = nn.Linear(starter*16,1)
        self.activation = activation
        
    def forward(self,X):
        preds = self.activation(self.fc1batchnorm(self.fc1(X)))
        preds = self.activation(self.fc2batchnorm(self.fc2(preds)))
        preds = self.activation(self.fc3batchnorm(self.fc3(preds)))
        preds = self.activation(self.fc4batchnorm(self.fc4(preds)))
        preds = self.activation(self.fc5batchnorm(self.fc5(preds)))
        preds = self.activation(self.fc6batchnorm(self.fc6(preds)))
        preds = self.activation(self.fc7batchnorm(self.fc7(preds)))
        preds = self.output(preds)
        preds = torch.sigmoid(preds)
        return preds

In [48]:
PROJECT_NAME = 'Titanic-V3'

In [49]:
def train(config,tune_using=True,name='test'):
    if tune_using is False:
        wandb.init(project=PROJECT_NAME,name=name)
    epochs = config['epochs']
    batch_size = config['batch_size']
    criterion = config['criterion']()
    model = Model(starter=config['starter'],activation=config['activation']).to(device)
    optimizer = config['optimizer'](model.parameters(),lr=config['lr'])
    for epoch in tqdm(range(epochs)):
        for idx in range(0,len(X_train),batch_size):
            X_batch = torch.tensor(np.array(X_train)[idx:idx+batch_size]).to(device)
            y_batch = torch.tensor(np.array(y_train)[idx:idx+batch_size]).to(device)
            model.to(device)
            preds = model(X_batch.float()).to(device)
            loss = criterion(preds.float(),y_batch.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            wandb.log({'loss':loss.item()})
        wandb.log({'val_accuracy':test(X_test,y_test,model)})
#             tune.report(mean_loss=loss.item(),accuracy=test(X_test,y_test,model))
    return model

In [50]:
# config = {'epochs':100,'batch_size':32,'criterion':nn.MSELoss,'starter':64,'activation':nn.ReLU(),'optimizer':optim.Adam,'lr':0.0001}
# train(config,False,'baseline-with-fe-2')

In [51]:
# config = {'epochs':100,'batch_size':32,'criterion':nn.MSELoss,'starter':64,'activation':nn.ReLU(),'optimizer':optim.Adam,'lr':0.001}
# model = train(config,False,'baseline-with-fe-2')

In [52]:
# test(X_test,y_test,model)

In [53]:
analysis = tune.run(
    train,
    config={
        "epochs": tune.grid_search([12]),
        "batch_size": tune.grid_search([16,32,64,128,256]),
        "criterion": tune.grid_search([nn.BCELoss]),
        "starter": tune.grid_search([32,64,128,256]),
        "activation": tune.grid_search([nn.LeakyReLU(),nn.PReLU(),nn.ReLU(),nn.ReLU6(),nn.RReLU(),nn.ELU(),nn.SiLU()]),
        'optimizer':tune.grid_search([optim.Adam,optim.AdamW,optim.Adamax,optim.Adagrad]),
        'lr':tune.grid_search([1e-3,1e-4])
    },resources_per_trial={"gpu": 1},)
print("Best config: ", analysis.get_best_config(
    metric="mean_loss", mode="min"))
print("Best config: ", analysis.get_best_config(
    metric="accuracy", mode="max"))
print("Best config: ", analysis.get_best_config(
    metric="accuracy", mode="min"))

In [54]:
print("Best config: ", analysis.get_best_config(
    metric="mean_loss", mode="min"))
print('\n')
print("Best config: ", analysis.get_best_config(
    metric="accuracy", mode="max"))

print("Best config: ", analysis.get_best_config(
    metric="accuracy", mode="min"))
print('\n')

NameError: name 'analysis' is not defined

In [55]:
df = analysis.results_df
df.head()
df.to_csv('./logs.csv',index=False)

NameError: name 'analysis' is not defined

In [56]:
# config = analysis.get_best_config(metric="mean_loss", mode="min")
# config['epochs'] = 250
# model = train(config,False,f'final-{config["epochs"]}-mean_loss')

In [57]:
config = analysis.get_best_config(metric="accuracy", mode="max")
config['epochs'] = 1000
model = train(config,False,f'final-{config["epochs"]}-accuracy-max')

NameError: name 'analysis' is not defined

In [326]:
# config = analysis.get_best_config(metric="accuracy", mode="min")
# config['epochs'] = 250
# model = train(config,False,f'final-{config["epochs"]}-accuracy-min')

loss,0.00224
_runtime,19
_timestamp,1625631307
_step,1499
val_accuracy,0.545


loss,█▅▅▂▂▂▂▃▂▁▁▂▂▁▁▁▁▃▁▁▁▁▂▁▁▁▁▂▁▁▁▁▂▁▁▁▁▁▁▁
_runtime,▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇█
_timestamp,▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇█
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_accuracy,▂▂▂▅▄█▅▁▄█▇█▇▂▇▇▇▅▂▄▅▅▇▅▅▅▇▅▅▅▅▄▅▄▅▄▅▄▄▄


100%|██████████| 250/250 [00:14<00:00, 16.93it/s]


In [327]:
test_df = pd.read_csv('./test.csv')

In [328]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [329]:
test_df.drop('Name',axis=1,inplace=True)

In [330]:
test_df.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,female,22.0,1,1,3101298,12.2875,NaN,S


In [332]:
test_df.drop('Cabin',axis=1,inplace=True)

In [333]:
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
53,54,1,2,1,29.0,1,0,2926,26.00,1
56,57,1,2,1,21.0,0,0,31026,10.50,1
58,59,1,2,1,5.0,1,2,34651,27.75,1
66,67,1,2,1,29.0,0,0,29395,10.50,1
70,71,0,2,0,32.0,0,0,33111,10.50,0


In [334]:
test_df.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,3,male,34.5,0,0,330911,7.8292,Q
1,893,3,female,47.0,1,0,363272,7.0000,S
2,894,2,male,62.0,0,0,240276,9.6875,Q
3,895,3,male,27.0,0,0,315154,8.6625,S
4,896,3,female,22.0,1,1,3101298,12.2875,S


In [335]:
labels = {'male':0,'female':1}

In [336]:
all = []
for iter in test_df['Sex']:
        all.append(labels[iter])

In [337]:
test_df['Sex'] = all

In [338]:
labels = {'S':0,'C':1,'Q':2}

In [339]:
all = []
for iter in test_df['Embarked']:
        all.append(labels[iter])

In [340]:
test_df['Embarked'] = all

In [341]:
test_df

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,3,0,34.5,0,0,330911,7.8292,2
1,893,3,1,47.0,1,0,363272,7.0000,0
2,894,2,0,62.0,0,0,240276,9.6875,2
3,895,3,0,27.0,0,0,315154,8.6625,0
4,896,3,1,22.0,1,1,3101298,12.2875,0
...,...,...,...,...,...,...,...,...,...
413,1305,3,0,NaN,0,0,A.5. 3236,8.0500,0
414,1306,1,1,39.0,0,0,PC 17758,108.9000,1
415,1307,3,0,38.5,0,0,SOTON/O.Q. 3101262,7.2500,0
416,1308,3,0,NaN,0,0,359309,8.0500,0


In [342]:
ids = test_df['PassengerId']

In [343]:
tickets = []
for ticket in test_df['Ticket']:
    ticket_splited = ticket.split(' ')
    try:
        ticket = ticket_splited[1]
    except:
        ticket = ticket_splited[0]
    try:
        tickets.append(int(ticket))
    except:
        try:
            ticket = ticket_splited[2]
        except:
            ticket = np.mean(tickets)
        tickets.append(int(ticket))

In [344]:
test_df['Ticket'] = tickets

In [345]:
test_df = np.array(test_df)

In [346]:
preds = []
model.eval()
with torch.no_grad():
    for idx in range(len(test_df)):
        pred = model(torch.tensor(test_df[idx]).to(device).float().view(1,9))
        preds.append(pred)

In [347]:
import random

In [348]:
preds_new = []
for pred in preds:
    try:
        preds_new.append(round(float(np.array(pred[0][0].to('cpu')))))
    except:
        preds_new.append(random.choice([0,1]))

In [349]:
submission = pd.DataFrame({'PassengerId':ids,'Survived':preds_new})

In [350]:
sample_submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [351]:
submission.to_csv('sub-0.csv',index=False)